In [1]:
import pandas as pd 
import numpy as np
import csv
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

/home/manasa/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [10]:
def getXY(filename):
  X = pd.read_csv(filename)
  if(filename=='merged.csv'):
        X = X.iloc[:,564:]

  x_drop = ["LONGITUDE", "LATITUDE", "FLOOR", "BUILDINGID", "SPACEID", "RELATIVEPOSITION", "USERID", "PHONEID", "TIMESTAMP"]
  y_cols = ["LONGITUDE", "LATITUDE","FLOOR","BUILDINGID"]

  x_cols = list(set(X.columns)-set(x_drop))
  X_train = X[x_cols]
  X_train.iloc[:, 0:520] = np.where(X_train.iloc[:, 0:520] <= 0, X_train.iloc[:, 0:520] + 105, X_train.iloc[:, 0:520] - 100)

  Y_train = X[y_cols]
  print(Y_train["LONGITUDE"].min())
  print(Y_train["LATITUDE"].min())

  Y_train["LONGITUDE"] += 7700
  Y_train["LATITUDE"] -= 4864745

  return X_train, Y_train

#X_train, Y_train = getXY('TrainingData.csv')
#print(X_train)
#print(Y_train)

full_input = pd.read_csv('merged.csv')
act_input = full_input.iloc[:,1:562]
act_true = full_input.iloc[:,563]

loc_input, loc_true = getXY('merged.csv')


-7691.109200000763
4864754.248742422


In [8]:
X,Y = getXY('ValidationData.csv')


-7695.93875492993
4864748.01548531


In [9]:
def pr(x):
    for i in range(len(x)):
        if x[i]!=0:
            print(i,x[i])
pr(X.iloc[0,:])
pr(X_test.iloc[0,:])

207 14


NameError: name 'X_test' is not defined

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier as rf
import pickle


y_cols = ["LONGITUDE", "LATITUDE","FLOOR","BUILDINGID"]
longitude_model = RandomForestRegressor()
longitude_model.fit(X_train, Y_train[y_cols[0]])
pickle.dump(longitude_model, open('longitude_model.sav', 'wb'))

latitude_model = RandomForestRegressor()
latitude_model.fit(X_train, Y_train[y_cols[1]])
pickle.dump(latitude_model, open('latitude_model.sav', 'wb'))

floor_model = rf()
floor_model.fit(X_train, Y_train[y_cols[2]])
pickle.dump(floor_model, open('floor_model.sav', 'wb'))

building_model = rf()
building_model.fit(X_train, Y_train[y_cols[3]])
pickle.dump(building_model, open('building_model.sav', 'wb'))

'''

'''

NameError: name 'X_train' is not defined

In [7]:
import pickle
activity_model = 0
latitude_model = 0
longitude_model = 0
floor_model = 0
building_model = 0


activity_model = pickle.load(open('activity_knn.sav', 'rb'))
latitude_model = pickle.load(open('latitude_model.sav', 'rb'))
longitude_model = pickle.load(open('longitude_model.sav', 'rb'))
floor_model = pickle.load(open('floor_model.sav', 'rb'))
building_model = pickle.load(open('building_model.sav', 'rb'))


In [8]:
def predict(user_id):
    pred = [user_id]
    activity_input = (np.array(act_input.iloc[user_id,:])).reshape(1,-1)
    pred.append(activity_model.predict(activity_input))
    
    localisation_input = (np.array(loc_input.iloc[user_id,:])).reshape(1,-1)
    pred.append(floor_model.predict(localisation_input))
    pred.append(building_model.predict(localisation_input))
    pred.append(latitude_model.predict(localisation_input))
    pred.append(longitude_model.predict(localisation_input))
    
    #print("User: {} {} on Floor: {} in Building: {} Location Coordinates : {},{}".format(pred[0], pred[1][0],pred[2][0],pred[3][0],pred[4][0],pred[5][0])) 
    #print("G.Truth {} on Floor: {} in Building: {} Location Coordinates : {},{}\n".format(act_true[user_id],loc_true.iloc[user_id,2],loc_true.iloc[user_id,3],loc_true.iloc[user_id,1],loc_true.iloc[user_id,0])) 
    # print(pred)
    return pred

    

In [6]:

fa = 0
ba = 0
#n = len(X_test)
n=200

for i in range(0,n):
    pred = predict(i)
    if(pred[2] == loc_true.iloc[i,2]):
        fa += 1/n
    if(pred[3] == loc_true.iloc[i,3]):
        ba += 1/n
    
print(fa)
print(ba)

AttributeError: 'int' object has no attribute 'predict'

In [11]:
fa = 0
ba = 0
#n = len(X_test)
n=200

for i in range(0,n):
    pred = predict(i)
    if(pred[1] == act_true.iloc[i]):
        fa += 1/n
    
print(fa)

0.9200000000000007


In [85]:
def read_user_id():
    print("Enter User Id b'n 0-199:")
    user_id = int(input())
    while user_id<0 or user_id>=200:
        print("Enter User Id b'n 0-199:")
        user_id = int(input())
    return user_id



In [87]:
predict(read_user_id())

Enter User Id b'n 0-199:
5
User: 5 STANDING on Floor: 2 in Building: 2 Location Coordinates : 79.451486806795,362.5132080000639
G.Truth STANDING on Floor: 2 in Building: 2 Location Coordinates : 80.46686277911067,361.19278958122595



[5,
 array(['STANDING'], dtype=object),
 array([2]),
 array([2]),
 array([79.45148681]),
 array([362.513208])]